In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import sys
sys.path.append('../')

from imc.matching_network import Theory

In [ ]:
# try to plot the Fig. 6 from the article
#
# this figure corresponds to a coarse tuning
# so, here the formula for |Γ| is a bit different
# than (4) which is for fine tuning only

# the parameters below are taken from the article
COMPLEX_ONE = complex(1, 0)
OMEGA = 2 * math.pi * 13.56e6
ZO = complex(50)

# ranges of variable capacitors from Fig. 6
cl_min=600e-12
cl_max=1600e-12
ct_min=50e-12
ct_max=250e-12

# my arbitrary choice
step=25e-12

def gamma(cl, ct):
    zl = COMPLEX_ONE / complex(0, OMEGA * cl)
    zt = COMPLEX_ONE / complex(0, OMEGA * ct)
    zy = zl * (zt + ZPL) / (zl + zt + ZPL)
    gamma = (zy - ZO) / (zy + ZO)
    return gamma

def gamma_modulus(cl, ct):
    return abs(gamma(cl, ct))

Res = np.arange(0.0, 200.0, 1.0)
Ims = np.arange(-200.0, 200.0, 1.0)
for ZPL_Re in np.nditer(Res):
    for ZPL_Im in np.nditer(Ims):
        ZPL = complex(ZPL_Re, ZPL_Im)
         
        x = np.arange(cl_min, cl_max, step)
        y = np.arange(ct_min, ct_max, step)

        X, Y = np.meshgrid(x, y)
        x = np.ravel(X)
        y = np.ravel(Y)

        z = np.array(list(map(gamma_modulus, x, y)))
        Z = z.reshape(X.shape)

        # skip not interesting figures
        min_z_modulus = np.amin(Z)
        min_index = np.argmin(Z)
        if min_z_modulus < 0.01: 
            # create figure
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')

            ax.plot_surface(X, Y, Z)

            ax.set_xlabel('Cl')
            ax.set_ylabel('Ct')
            #ax.set_zlabel('|Γ|')
            ax.set_title("ZPL=(" + str(ZPL_Re) + ',' + str(ZPL_Im) + ")"
                         + "  min(|Γ|)=" + str(np.around(min_z_modulus, decimals=2))
                         + "  Cl=" + str(np.format_float_scientific(x[min_index], precision=2))
                         + "  Ct=" + str(np.format_float_scientific(y[min_index], precision=2))
                        )
            
            plt.show()


In [ ]:
# for example, this value of ZPL makes sense for me
# ZPL=(5.0, 109.0)  min(|Γ|)=0.01  Cl=700pF  Ct=125pF
# I will use it in the experimental code
# together with slightly disturbed capacitors